<a href="https://colab.research.google.com/github/1900690/gradio/blob/main/gradio_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[Google Colab上でwebアプリが作れるGradioの使い方](https://clameyes.com/posts/google-colab-web-app-gradio)

[garadioのplaygroundにサンプルがたくさん](https://www.gradio.app/playground)

##※認証がある場合はID;test pass:gradio

In [ ]:
#@title gradioライブラリをインストール
#@markdown この共有リンクの有効期限は1週間です。永続的なホスティングとGPUのアップグレードを無料で行うには、作業ディレクトリのターミナルから `gradio deploy` を実行し、Hugging Face Spaces (https://huggingface.co/spaces) にデプロイしてください。
!pip -q install gradio

In [ ]:
#@title 指定したテキストを返すアプリ
import gradio as gr

# 関数を定義
def greet(name):
    return "Hello " + name + "!"

# Webアプリを作成
app = gr.Interface(fn=greet, inputs="text", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2db3a0419addaceb0.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#@title スライダーで数を定義するアプリ
import gradio as gr

# 関数を定義
def count(number):
    return "数字は " + str(number) + "!"

# Webアプリを作成
app = gr.Interface(fn=count, inputs="slider", outputs="text")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1d38daa5056d6f920f.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[Google ColabでGUI｜gradioを利用したウェブアプリの作り方を解説](https://tech.aru-zakki.com/gradio-on-google-colab/)

In [ ]:
#@title 画像をフィルターに通すアプリ
import numpy as np
import gradio as gr

#セピアフィルターを定義
def sepia(input_img):
    sepia_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    sepia_img = input_img.dot(sepia_filter.T)
    sepia_img /= sepia_img.max()
    return sepia_img

app = gr.Interface(sepia, gr.Image(), "image")
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://402e2f7432d4e668b6.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
#@title　入力した値を覚えるアプリ
import gradio as gr

scores = []

def track_score(score):
    scores.append(score)
    top_scores = sorted(scores, reverse=True)[:3]
    return top_scores

app = gr.Interface(
    track_score,
    gr.Number(label="Score"),
    gr.JSON(label="Top Scores")
)
app.launch(share=True, auth=("test", "gradio"))

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://cd3b70f4d607c1eddb.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#アナログメータ針検出アプリをgradio化してみると

In [ ]:
import cv2
import math
import numpy as np
import gradio as gr

def analog(input_img):
        # 指定する中心点（メータの中心）
        cx, cy =(346,304)#横は→に進む、縦は↓に進む
        angle_off_set=-133#pf0の時に０になるように初期値を設定する

        # グレースケール変換 & 二値化
        gray = cv2.cvtColor(input_img, cv2.COLOR_BGR2GRAY)
        threshold = 100
        _, img_thresh = cv2.threshold(gray, threshold, 255, cv2.THRESH_BINARY)

        # エッジ検出
        edges = cv2.Canny(img_thresh, 50, 200, apertureSize=3)

        # Hough変換で直線を検出（2本の直線を得る）
        minn = None
        for m in range(10, 161, 1):
            lines = cv2.HoughLines(edges, 1, np.pi / 180, m)
            if lines is not None and len(lines) == 2:
                minn = m
                break

        # 直線が2本見つからなかった場合のエラーハンドリング
        if minn is None:
            raise ValueError("直線が2本検出されませんでした。画像のコントラストやエッジ検出のパラメータを調整してください。")

        lines = cv2.HoughLines(edges, 1, np.pi / 180, minn)

        theta_t = []  # 直線の角度
        coefficients = []  # 直線の傾きと切片

        for i in range(len(lines)):
            for rho, theta in lines[i]:
                theta_t.append(theta)

                a = np.cos(theta)
                b = np.sin(theta)
                x0 = a * rho
                y0 = b * rho

                x1 = int(x0 + 1000 * (-b))
                y1 = int(y0 + 1000 * (a))
                x2 = int(x0 - 1000 * (-b))
                y2 = int(y0 - 1000 * (a))
                cv2.line(input_img, (x1, y1), (x2, y2), (0, 0, 255), 2)

                # x2-x1が0の場合のエラーハンドリング
                if x2 - x1 == 0:
                    continue  # 無限傾斜を避ける
                a0 = (y2 - y1) / (x2 - x1)
                b0 = y1 - a0 * x1
                coefficients.append((a0, b0))

        # 直線の数をチェック（2本以上ある場合は最初の2本を使用）
        if len(coefficients) < 2:
            raise gr.Error("十分な直線が検出されませんでした。")

        (a1, b1), (a2, b2) = coefficients[:2]  # 最初の2本を選択

        # 交点を計算
        if a1 == a2:
            raise gr.Error("平行な直線のため交点が計算できません。")

        x_t = (b2 - b1) / (a1 - a2)
        y_t = a1 * x_t + b1

        # 交点と中心点を結ぶ線の角度を計算
        angle_rad = np.arctan2(y_t - cy, x_t - cx)
        angle_deg = np.degrees(angle_rad)
        angle =int(angle_deg) +angle_off_set
        angle=angle*-1  #負の角度を修正
        Kpa=8/27*angle
        if Kpa * 10.197 > 0:
          PH=math.log10(Kpa*10.197)
        elif Kpa == 0 :
          PH=0
        else:
          PH=-1
          raise gr.Error("角度がマイナスになりました")

        # 結果を画像に描画
        img_temp=cv2.circle(input_img, (int(cx), int(cy)), 10, (0, 255, 0), -1)  # 中心点を描画
        img_temp=cv2.circle(img_temp, (int(x_t), int(y_t)), 10, (0, 255, 0), -1)  # 交点を描画
        img_temp=cv2.line(img_temp, (cx, cy), (int(x_t), int(y_t)), (255, 0, 0), 2)  # 中心と交点を結ぶ線
        img_temp=cv2.putText(img_temp, f"{PH:.2f} ph", (100, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        return img_temp

app = gr.Interface(
    analog,
    gr.Image(),
    "image",
     examples=[
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/000.jpg"],
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/021.jpg"],
         ["https://github.com/1900690/analog-meter-reading/releases/download/analog-meter-samples/026.jpg"]
     ],
    title="アナログメータ針検出アプリ",
    description="アナログメーターの画像をアップロードしてください。画像からから針の角度を検出します。",
    allow_flagging='never'
    )
# Webアプリを起動
app.launch(share=True, auth=("test", "gradio"))

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d1d3ce4eeec00673aa.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#空隙率測定アプリをgradio化してみると

In [ ]:
import cv2
import numpy as np
import gradio as gr

def analog(input_img):
        #画像を読み込み
        src = cv2.cvtColor(input_img, cv2.COLOR_BGR2RGB)
        #各色を取り出し
        r = np.array(src[:, :, 0], dtype=np.int16)
        g = np.array(src[:, :, 1], dtype=np.int16)
        b = np.array(src[:, :, 2], dtype=np.int16)

        return b

gr.Slider(minimum=1, maximum=10, step=1),

app = gr.Interface(
    analog,#関数名
    [gr.Image(),#入力する値
     gr.Slider(minimum=0, maximum=255, step=1, label="赤上限"),
     gr.Slider(minimum=0, maximum=255, step=1, label="赤下限"),
     gr.Slider(minimum=0, maximum=255, step=1, label="青上限"),
     gr.Slider(minimum=0, maximum=255, step=1, label="青下限"),
     gr.Slider(minimum=0, maximum=255, step=1, label="緑上限"),
     gr.Slider(minimum=0, maximum=255, step=1, label="緑下限"),
     ],
    "image",#出力する値
     examples=[#例の入力
         ["https://github.com/1900690/grape-sky-segmentation/releases/download/sample/sample.jpg"]
     ],
    allow_flagging='never'
    )
# Webアプリを起動
app.launch()

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:416: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://92ef2ebee7fdba13b9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
